# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [1]:
from utils import *
from config import *
import pandas as pd
from datetime import datetime

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

## Class Definitions

In [ ]:
def main(wdpaid, year, show_map=False, band_name=None):
    """Main function to process protected area boundary analysis"""
    # Initialize classes
    geo_ops = GeometryOperations()
    img_ops = ImageOperations()
    stats_ops = StatsOperations()
    viz = Visualization()
    feature_processor = FeatureProcessor(geo_ops, img_ops, stats_ops)
    exporter = ExportResults()

    # Load and process protected area geometry
    pa = load_local_data(wdpaid)
    pa_geometry = pa.geometry()
    aoi = geo_ops.buffer_polygon(pa_geometry)
    aoi = geo_ops.mask_water(aoi)
    aoi_with_biome = geo_ops.get_biome(aoi)

    # Process imagery and add indices
    modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi, year))
    band_names = modis_ic.first().bandNames()
    composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
    image = img_ops.add_indices_to_image(composite)

    # Process features and collect statistics
    feature_info = feature_processor.collect_feature_info(pa, aoi_with_biome)
    computed_stats = feature_processor.process_all_bands(image, pa_geometry)
    all_stats = feature_processor.compile_statistics(feature_info, computed_stats, year)
    
    # Save results
    #df, _ = exporter.save_statistics_to_csv(all_stats, wdpaid, year)
    df = pd.DataFrame(all_stats)
    exporter.save_df_to_gcs(df, 'dse-staff', wdpaid, year)

    # Visualization
    if show_map:
        band_stats = next(cs for cs in computed_stats if cs["band_name"] == band_name)
        Map = viz.create_map(pa_geometry, band_stats['gradient'], band_stats['boundary_pixels'])
    
    return Map

In [ ]:
Map = main("916", 2010, show_map=True, band_name="EVI")
Map

In [ ]:
try:
    # Load and verify geometry
    pa = load_local_data("367731")
    pa2 = load_protected_area("367731")
    print("1. Successfully loaded protected area")
    
    # Create map with debug prints
    import geemap
    Map = geemap.Map()
    print("2. Created base map")
    
    # Add basemap and geometry with error checking
    try:
        Map.add_basemap('HYBRID')
        Map.addLayer(
            ee.FeatureCollection([pa]), 
            {'color': 'red', 'fillColor': '#ff000033', 'width': 2},
            'Protected Area'
        )
        Map.addLayer(
            ee.FeatureCollection([pa2]), 
            {'color': 'blue', 'fillColor': '#ff000033', 'width': 2},
            'Protected Area2'
        )
        bounds = pa.geometry().bounds()
        Map.centerObject(bounds, zoom=8)
        display(Map)
        print("3. Map displayed successfully")
        
    except Exception as e:
        print(f"Map visualization error: {e}")
        
except Exception as e:
    print(f"Error: {e}")
    if 'pa' in locals():
        print("\nDebug info:")
        print(f"Feature type: {type(pa)}")
        print(f"Properties: {pa.propertyNames().getInfo()}")


-add write out for each step, with identifier for each park
-ray to run in parallel in python
-use glance to check usage